# Data Acquisition, Cleaning, and Integration
### Project: Mapping and Modeling Drug-Related Deaths in Cook County

This notebook demonstrates interim progress on data acquisition, cleaning, and integration. There is secondary notebook that will conduct EDA.  
It documents the steps taken so far to prepare datasets for spatial and demographic analysis, as outlined in our Project Plan.


In [1]:
# imports

import pandas as pd
import requests
import hashlib
import os

from pathlib import Path

from textblob import TextBlob


In [2]:
# Move working directory to project root
project_root = Path(".").resolve().parent 
os.chdir(project_root)

print("Current working directory:", os.getcwd())

Current working directory: /Users/aaliyahali/Desktop/is477/IS477-FinalProject


In [3]:
#!/usr/bin/env python3

import pandas as pd
import os
from pathlib import Path

# Ensure output directory exists
os.makedirs("data/processed", exist_ok=True)

# ---- Load cleaned ME data ----
me_path = Path("data/processed/me_cleaned.csv")
df_me = pd.read_csv(me_path, dtype={"Incident Zip Code": str})

# ---- Load ZIP → TRACT Crosswalk ----
crosswalk_path = Path("data/raw/ZIP_TRACT_062025.xlsx")
df_zip = pd.read_excel(crosswalk_path, dtype={"ZIP": str, "TRACT": str})

# ---- Load cleaned Census tract dataset ----
census_path = Path("data/processed/census_data_cleaned.csv")
df_census = pd.read_csv(census_path, dtype={"TRACT_FIPS": str})


/var/folders/m9/vbpb06vj43vdgzjt4xlxm9rm0000gn/T/ipykernel_61687/3646030261.py:12: DtypeWarning: Columns (25,29) have mixed types. Specify dtype option on import or set low_memory=False.
  df_me = pd.read_csv(me_path, dtype={"Incident Zip Code": str})


In [4]:
df_zip.columns

Index(['ZIP', 'TRACT', 'USPS_ZIP_PREF_CITY', 'USPS_ZIP_PREF_STATE',
       'RES_RATIO', 'BUS_RATIO', 'OTH_RATIO', 'TOT_RATIO'],
      dtype='object')

In [5]:
# ---- Filter Crosswalk to Cook County ----
df_zip_cook = df_zip[df_zip["TRACT"].str.startswith("17031")].copy()

# ---- Pick the primary tract per ZIP (highest residential share) ----
df_zip_primary = (
    df_zip_cook
    .sort_values(["ZIP", "RES_RATIO"], ascending=[True, False])
    .drop_duplicates(subset="ZIP")
    .reset_index(drop=True)
)

print("Original crosswalk row count:", len(df_zip))
print("Cook only:", len(df_zip_cook))
print("Primary ZIP→Tract:", len(df_zip_primary))


Original crosswalk row count: 189302
Cook only: 2217
Primary ZIP→Tract: 228


In [6]:
df_merge = df_me.merge(
    df_zip_primary,
    how="left",
    left_on="Incident Zip Code",
    right_on="ZIP"
)

print("Rows after ZIP→TRACT merge:", len(df_merge))
# should be ~51791 rows (not millions)


Rows after ZIP→TRACT merge: 51791


In [7]:
df_merge = df_merge.merge(
    df_census,
    how="left",
    left_on="TRACT",
    right_on="TRACT_FIPS"
)

print("Final row count:", len(df_merge))


Final row count: 51791


In [8]:
df_merge.columns

Index(['Case Number', 'Date of Incident', 'Date of Death', 'Age', 'Gender',
       'Race', 'Latino', 'Manner of Death', 'Primary Cause',
       'Primary Cause Line A', 'Primary Cause Line B', 'Primary Cause Line C',
       'Secondary Cause', 'Gun Related', 'Opioid Related', 'Cold Related',
       'Heat Related', 'Commissioner District', 'Incident Address',
       'Incident City', 'Incident Zip Code', 'longitude', 'latitude',
       'location', 'Residence City', 'Residence Zip', 'OBJECTID',
       'Chicago Ward', 'Chicago Community Area', 'COVID Related',
       'Updated_Date_of_Death', 'Year_of_Death', 'contanins_fentanyl',
       'contains_cocaine', 'contains_heroin', 'ZIP', 'TRACT_x',
       'USPS_ZIP_PREF_CITY', 'USPS_ZIP_PREF_STATE', 'RES_RATIO', 'BUS_RATIO',
       'OTH_RATIO', 'TOT_RATIO', 'NAME', 'TotalPop', 'White_NonHisp', 'Black',
       'Asian', 'Hispanic', 'MedianIncome', 'BelowPoverty',
       'BachelorsOrHigher', 'Male_Under18', 'Female_Under18', 'Male_18_34',
       'Fem

In [9]:
# To display all columns
pd.set_option('display.max_columns', None)

# To display all rows
pd.set_option('display.max_rows', None)
df_merge.isna().mean()

Case Number               0.000000
Date of Incident          0.024444
Date of Death             0.000000
Age                       0.003089
Gender                    0.002047
Race                      0.004171
Latino                    0.000000
Manner of Death           0.004653
Primary Cause             0.000270
Primary Cause Line A      0.000270
Primary Cause Line B      0.644649
Primary Cause Line C      0.948060
Secondary Cause           0.567840
Gun Related               0.004692
Opioid Related            0.004692
Cold Related              0.000000
Heat Related              0.000000
Commissioner District     0.109208
Incident Address          0.002993
Incident City             0.001506
Incident Zip Code         0.000000
longitude                 0.109112
latitude                  0.109112
location                  0.109112
Residence City            0.020505
Residence Zip             0.020293
OBJECTID                  0.000000
Chicago Ward              0.440926
Chicago Community Ar

In [10]:
# To display all columns
pd.set_option('display.max_columns', None)

# To display all rows
pd.set_option('display.max_rows', None)

df_merge.isna().sum()

Case Number                   0
Date of Incident           1266
Date of Death                 0
Age                         160
Gender                      106
Race                        216
Latino                        0
Manner of Death             241
Primary Cause                14
Primary Cause Line A         14
Primary Cause Line B      33387
Primary Cause Line C      49101
Secondary Cause           29409
Gun Related                 243
Opioid Related              243
Cold Related                  0
Heat Related                  0
Commissioner District      5656
Incident Address            155
Incident City                78
Incident Zip Code             0
longitude                  5651
latitude                   5651
location                   5651
Residence City             1062
Residence Zip              1051
OBJECTID                      0
Chicago Ward              22836
Chicago Community Area    22836
COVID Related                 2
Updated_Date_of_Death         0
Year_of_

In [11]:
# Standardize tract column name
df_merge["TRACT"] = df_merge["TRACT_x"]

# Drop duplicate/unneeded columns
df_merge = df_merge.drop(columns=[
    "TRACT_x", "TRACT_y", "TRACT_FIPS",
    "USPS_ZIP_PREF_CITY", "USPS_ZIP_PREF_STATE",
    "BUS_RATIO", "OTH_RATIO", "TOT_RATIO"
], errors="ignore")

df_merge.head()

,Case Number,Date of Incident,Date of Death,Age,Gender,Race,Latino,Manner of Death,Primary Cause,Primary Cause Line A,Primary Cause Line B,Primary Cause Line C,Secondary Cause,Gun Related,Opioid Related,Cold Related,Heat Related,Commissioner District,Incident Address,Incident City,Incident Zip Code,longitude,latitude,location,Residence City,Residence Zip,OBJECTID,Chicago Ward,Chicago Community Area,COVID Related,Updated_Date_of_Death,Year_of_Death,contanins_fentanyl,contains_cocaine,contains_heroin,ZIP,RES_RATIO,NAME,TotalPop,White_NonHisp,Black,Asian,Hispanic,MedianIncome,BelowPoverty,BachelorsOrHigher,Male_Under18,Female_Under18,Male_18_34,Female_18_34,Male_35_64,Female_35_64,Male_65plus,Female_65plus,LaborForce,Unemployed,NoHealthInsurance,MedianRent,STATE,COUNTY,Age_Under18,Age_18_34,Age_35_64,Age_65plus,Pct_Black,Pct_Hispanic,Pct_Asian,Pct_WhiteNonHisp,Pct_BelowPoverty,Pct_BachelorsPlus,Pct_Unemployed,Pct_Uninsured,TRACT
0,ME2025-02283,04/21/2023 02:08:00 PM,07/25/2023 12:00:00 AM,42.0,Male,Black,False,ACCIDENT,COMPLICATIONS OF DROWNING,COMPLICATIONS OF DROWNING,NaN,NaN,"HYPERTENSIVE CARDIOVASCULAR DISEASE, CHRONIC S...",False,False,False,False,1.0,100 N Central Park Avenue,CHICAGO,60624,-87.716075,41.882688,"(41.882688, -87.7160745)",Elmwood Park,60707.0,89165,28.0,EAST GARFIELD PARK,False,2023-07-25 00:00:00,2023.0,False,False,False,60624,0.142876,Census Tract 2315; Cook County; Illinois,6183.0,332.0,3361.0,176.0,2281.0,31755.0,1956.0,311.0,242.0,551.0,87.0,137.0,0.0,139.0,25.0,39.0,2437.0,394.0,45.0,1133.0,17.0,31.0,793.0,224.0,139.0,64.0,54.358726,36.891477,2.846515,5.369562,31.635129,5.029921,16.167419,0.727802,17031231500
1,ME2025-01581,03/10/2025 12:00:00 AM,05/03/2023 11:36:00 PM,84.0,Male,Black,False,NATURAL,MALIGNANT NEOPLASM OF THE URINARY BLADDER,MALIGNANT NEOPLASM OF THE URINARY BLADDER,NaN,NaN,NaN,False,False,False,False,5.0,230 E. 11th St.,CHICAGO HEIGHTS,60411,-87.625580,41.511357,"(41.511357, -87.6255795)",NaN,NaN,88467,NaN,NaN,False,2023-05-03 23:36:00,2023.0,False,False,False,60411,0.107736,Census Tract 8285.08; Cook County; Illinois,4077.0,671.0,2396.0,0.0,744.0,29207.0,1159.0,343.0,291.0,180.0,44.0,0.0,42.0,319.0,114.0,55.0,1798.0,85.0,153.0,1075.0,17.0,31.0,471.0,44.0,361.0,169.0,58.768702,18.248712,0.000000,16.458180,28.427766,8.413049,4.727475,3.752759,17031828508
2,ME2025-01577,03/06/2022 08:11:00 PM,03/06/2022 07:20:00 PM,71.0,Male,White,True,NATURAL,CHRONIC ETHANOLISM,CHRONIC ETHANOLISM,NaN,NaN,DIABETES MELLITUS,False,False,False,False,7.0,2551 W. CERMAK ROAD,CHICAGO,60608,-87.690001,41.850558,"(41.850558, -87.6900015)",Chicago,60623.0,88470,28.0,SOUTH LAWNDALE,False,2022-03-06 19:20:00,2022.0,False,False,False,60608,0.082878,Census Tract 3106; Cook County; Illinois,5136.0,1533.0,228.0,349.0,2900.0,75604.0,583.0,1012.0,40.0,131.0,0.0,0.0,76.0,207.0,45.0,0.0,3040.0,143.0,0.0,1333.0,17.0,31.0,171.0,0.0,283.0,45.0,4.439252,56.464174,6.795171,29.848131,11.351246,19.704050,4.703947,0.000000,17031310600
3,ME2024-00980,11/30/2023 02:23:00 PM,12/01/2023 07:52:00 AM,87.0,Female,Asian,False,NATURAL,SEPSIS. BRONCHOPNEUMONIA,SEPSIS,BRONCHOPNEUMONIA,NaN,HYPERTENSIVE AND ATHEROSCLEROTIC CARDIOVASCULA...,False,False,False,False,NaN,8110 Lincoln Ave. #2C,CHICAGO,60646,NaN,NaN,NaN,Skokie,60077.0,80661,NaN,NaN,False,2023-12-01 07:52:00,2023.0,False,False,False,60646,0.219736,Census Tract 1203; Cook County; Illinois,7093.0,5562.0,21.0,602.0,598.0,137043.0,433.0,1783.0,329.0,249.0,40.0,19.0,136.0,55.0,171.0,48.0,3446.0,93.0,82.0,1368.0,17.0,31.0,578.0,59.0,191.0,219.0,0.296067,8.430847,8.487241,78.415339,6.104610,25.137459,2.698781,1.156069,17031120300
4,ME2024-00837,12/19/2023 02:42:00 PM,12/27/2023 11:03:00 PM,84.0,Male,Black,False,ACCIDENT,THERMAL AND INHALATION INJURIES. RESIDENTIAL F...,THERMAL AND INHALATION INJURIES,RESIDENTIAL FIRE,MISUSE OF SMOKING MATERIAL,NaN,False,False,False,False,10.0,4645 N Sheridan Rd Apt 407,CHICAGO,60640,-87.654168,41.966649,"(41.966649, -87.654168)",Chicago,60640.0,80512,46.0,UPT

In [12]:
# scripts/acquire_data.py

df_me = pd.read_csv("data/raw/Medical_Examiner_Case_Archive_20251104.csv", low_memory=False)

In [13]:
os.makedirs("data/raw", exist_ok=True)

# ACS 5-Year 2023 (data collected 2019–2023)
url = "https://api.census.gov/data/2023/acs/acs5"

variables = [
    # Population + race
    "B01003_001E",  # total population
    "B03002_003E",  # white (non-Hispanic)
    "B03002_004E",  # black
    "B03002_006E",  # asian
    "B03002_012E",  # hispanic

    # Income & poverty
    "B19013_001E",  # median income
    "B17001_002E",  # below poverty

    # Education
    "B15003_022E",  # bachelor's degree or higher

    # Age structure (male + female counts)
    "B01001_003E", "B01001_027E",  # under 18
    "B01001_007E", "B01001_031E",  # 18–34
    "B01001_010E", "B01001_034E",  # 35–64
    "B01001_020E", "B01001_044E",  # 65+

    # New: Unemployment, Health Insurance, Rent
    "B23025_002E",  # labor force
    "B23025_005E",  # unemployed
    "B27010_017E",  # no health insurance (all people)
    "B25064_001E"   # median gross rent
]

params = {
    "get": ",".join(["NAME"] + variables),
    "for": "tract:*",
    "in": "county:031 state:17"  # Cook County, IL
}

r = requests.get(url, params=params)
r.raise_for_status()

data = r.json()
cols = data[0]
rows = data[1:]
df_census = pd.DataFrame(rows, columns=cols)

# Rename columns for clarity
df_census = df_census.rename(columns={
    "B01003_001E": "TotalPop",
    "B03002_003E": "White_NonHisp",
    "B03002_004E": "Black",
    "B03002_006E": "Asian",
    "B03002_012E": "Hispanic",
    "B19013_001E": "MedianIncome",
    "B17001_002E": "BelowPoverty",
    "B15003_022E": "BachelorsOrHigher",
    "B01001_003E": "Male_Under18",
    "B01001_027E": "Female_Under18",
    "B01001_007E": "Male_18_34",
    "B01001_031E": "Female_18_34",
    "B01001_010E": "Male_35_64",
    "B01001_034E": "Female_35_64",
    "B01001_020E": "Male_65plus",
    "B01001_044E": "Female_65plus",
    "B23025_002E": "LaborForce",
    "B23025_005E": "Unemployed",
    "B27010_017E": "NoHealthInsurance",
    "B25064_001E": "MedianRent",
    "state": "STATE",
    "county": "COUNTY",
    "tract": "TRACT"
})

# Unique tract ID for merging
df_census["TRACT_FIPS"] = df_census["STATE"] + df_census["COUNTY"] + df_census["TRACT"].str.zfill(6)

df_census.to_csv("data/raw/census_tract_data.csv", index=False)
print(f"Saved enriched Census data for {len(df_census)} tracts.")


Saved enriched Census data for 1332 tracts.


In [14]:
df_census.head()

,NAME,TotalPop,White_NonHisp,Black,Asian,Hispanic,MedianIncome,BelowPoverty,BachelorsOrHigher,Male_Under18,Female_Under18,Male_18_34,Female_18_34,Male_35_64,Female_35_64,Male_65plus,Female_65plus,LaborForce,Unemployed,NoHealthInsurance,MedianRent,STATE,COUNTY,TRACT,TRACT_FIPS
0,Census Tract 101; Cook County; Illinois,3726,1297,1376,137,809,69460,508,775,88,41,52,141,128,46,108,56,2579,260,130,1252,17,031,010100,17031010100
1,Census Tract 102.01; Cook County; Illinois,7588,1406,2301,376,2622,49639,1892,684,455,353,43,62,161,317,143,119,4007,317,40,1333,17,031,010201,17031010201
2,Census Tract 102.02; Cook County; Illinois,2609,967,949,212,423,55119,524,656,9,32,48,0,43,90,24,31,1449,135,36,1292,17,031,010202,17031010202
3,Census Tract 103; Cook County; Illinois,6311,3094,1298,342,1426,65871,431,1585,148,25,33,35,53,179,26,84,3923,166,0,1257,17,031,010300,17031010300
4,Census Tract 104; Cook County; Illinois,4282,3173,296,324,340,49017,463,999,25,43,358,688,91,100,83,18,2282,78,16,1248,17,031,010400,17031010400


In [15]:
df_me.head()

,Case Number,Date of Incident,Date of Death,Age,Gender,Race,Latino,Manner of Death,Primary Cause,Primary Cause Line A,Primary Cause Line B,Primary Cause Line C,Secondary Cause,Gun Related,Opioid Related,Cold Related,Heat Related,Commissioner District,Incident Address,Incident City,Incident Zip Code,longitude,latitude,location,Residence City,Residence Zip,OBJECTID,Chicago Ward,Chicago Community Area,COVID Related
0,ME2025-05835,11/04/2025 02:09:00 PM,11/04/2025 01:57:00 PM,57.0,Male,Black,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,2.0,"901 WEST 63RD ST., UNIT 324",CHICAGO,60621,-87.647293,41.779635,"(41.77963481, -87.64729284)",Chicago,60621,92724,16.0,ENGLEWOOD,False
1,ME2025-05834,09/24/2025 01:48:00 PM,11/04/2025 12:20:00 PM,55.0,Female,White,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,7.0,4803 S LAFLIN ST FLOOR ONE UNIT,CHICAGO,60609,-87.662232,41.806638,"(41.806638, -87.662232)",Chicago,60609,92725,20.0,NEW CITY,False
2,ME2025-05833,11/04/2025 12:00:00 AM,11/04/2025 12:40:00 PM,69.0,Female,White,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,1.0,2737 W HIRSCH STREET 2ND FLR,CHICAGO,60622,-87.696022,41.906421,"(41.906421, -87.6960225)",Chicago,60612,92720,26.0,WEST TOWN,False
3,ME2025-05832,11/04/2025 01:12:00 PM,11/04/2025 12:40:00 PM,75.0,Male,White,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,1.0,2737 W HIRSCH STREET 2ND FLR,CHICAGO,60622,-87.696022,41.906421,"(41.906421, -87.6960225)",Chicago,60622,92719,26.0,WEST TOWN,False
4,ME2025-05831,11/04/2025 01:41:00 PM,11/04/2025 09:48:00 AM,30.0,Male,Black,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,5.0,1722 W. 170TH,HAZEL CREST,60429,-87.659707,41.581080,"(41.58108, -87.6597075)",Hazel Crest,60429,92722,NaN,NaN,False


### Exploratory Data Analysis 

In [16]:
len(df_me)

92642

In [17]:
df_me.dtypes

Case Number                object
Date of Incident           object
Date of Death              object
Age                       float64
Gender                     object
Race                       object
Latino                       bool
Manner of Death            object
Primary Cause              object
Primary Cause Line A       object
Primary Cause Line B       object
Primary Cause Line C       object
Secondary Cause            object
Gun Related                object
Opioid Related             object
Cold Related                 bool
Heat Related                 bool
Commissioner District     float64
Incident Address           object
Incident City              object
Incident Zip Code          object
longitude                 float64
latitude                  float64
location                   object
Residence City             object
Residence Zip              object
OBJECTID                    int64
Chicago Ward              float64
Chicago Community Area     object
COVID Related 

In [18]:
df_me.columns

Index(['Case Number', 'Date of Incident', 'Date of Death', 'Age', 'Gender',
       'Race', 'Latino', 'Manner of Death', 'Primary Cause',
       'Primary Cause Line A', 'Primary Cause Line B', 'Primary Cause Line C',
       'Secondary Cause', 'Gun Related', 'Opioid Related', 'Cold Related',
       'Heat Related', 'Commissioner District', 'Incident Address',
       'Incident City', 'Incident Zip Code', 'longitude', 'latitude',
       'location', 'Residence City', 'Residence Zip', 'OBJECTID',
       'Chicago Ward', 'Chicago Community Area', 'COVID Related'],
      dtype='object')

## Cleaning + Pre-Processing

### Census Data

In [19]:
df_census.isna().sum()
# no missing values

NAME                 0
TotalPop             0
White_NonHisp        0
Black                0
Asian                0
Hispanic             0
MedianIncome         0
BelowPoverty         0
BachelorsOrHigher    0
Male_Under18         0
Female_Under18       0
Male_18_34           0
Female_18_34         0
Male_35_64           0
Female_35_64         0
Male_65plus          0
Female_65plus        0
LaborForce           0
Unemployed           0
NoHealthInsurance    0
MedianRent           0
STATE                0
COUNTY               0
TRACT                0
TRACT_FIPS           0
dtype: int64

In [20]:
df_census.head()

,NAME,TotalPop,White_NonHisp,Black,Asian,Hispanic,MedianIncome,BelowPoverty,BachelorsOrHigher,Male_Under18,Female_Under18,Male_18_34,Female_18_34,Male_35_64,Female_35_64,Male_65plus,Female_65plus,LaborForce,Unemployed,NoHealthInsurance,MedianRent,STATE,COUNTY,TRACT,TRACT_FIPS
0,Census Tract 101; Cook County; Illinois,3726,1297,1376,137,809,69460,508,775,88,41,52,141,128,46,108,56,2579,260,130,1252,17,031,010100,17031010100
1,Census Tract 102.01; Cook County; Illinois,7588,1406,2301,376,2622,49639,1892,684,455,353,43,62,161,317,143,119,4007,317,40,1333,17,031,010201,17031010201
2,Census Tract 102.02; Cook County; Illinois,2609,967,949,212,423,55119,524,656,9,32,48,0,43,90,24,31,1449,135,36,1292,17,031,010202,17031010202
3,Census Tract 103; Cook County; Illinois,6311,3094,1298,342,1426,65871,431,1585,148,25,33,35,53,179,26,84,3923,166,0,1257,17,031,010300,17031010300
4,Census Tract 104; Cook County; Illinois,4282,3173,296,324,340,49017,463,999,25,43,358,688,91,100,83,18,2282,78,16,1248,17,031,010400,17031010400


In [21]:
df_census.dtypes

NAME                 object
TotalPop             object
White_NonHisp        object
Black                object
Asian                object
Hispanic             object
MedianIncome         object
BelowPoverty         object
BachelorsOrHigher    object
Male_Under18         object
Female_Under18       object
Male_18_34           object
Female_18_34         object
Male_35_64           object
Female_35_64         object
Male_65plus          object
Female_65plus        object
LaborForce           object
Unemployed           object
NoHealthInsurance    object
MedianRent           object
STATE                object
COUNTY               object
TRACT                object
TRACT_FIPS           object
dtype: object

In [22]:
# convert string columns to numeric values for computational purposes
num_cols = [c for c in df_census.columns if c not in ["NAME", "STATE", "COUNTY", "TRACT", "TRACT_FIPS"]] 
df_census[num_cols] = df_census[num_cols].apply(pd.to_numeric, errors="coerce")

In [23]:
df_census.dtypes

NAME                 object
TotalPop              int64
White_NonHisp         int64
Black                 int64
Asian                 int64
Hispanic              int64
MedianIncome          int64
BelowPoverty          int64
BachelorsOrHigher     int64
Male_Under18          int64
Female_Under18        int64
Male_18_34            int64
Female_18_34          int64
Male_35_64            int64
Female_35_64          int64
Male_65plus           int64
Female_65plus         int64
LaborForce            int64
Unemployed            int64
NoHealthInsurance     int64
MedianRent            int64
STATE                object
COUNTY               object
TRACT                object
TRACT_FIPS           object
dtype: object

In [24]:
# age group totals for analysis 
df_census["Age_Under18"] = df_census["Male_Under18"] + df_census["Female_Under18"]
df_census["Age_18_34"] = df_census["Male_18_34"] + df_census["Female_18_34"]
df_census["Age_35_64"] = df_census["Male_35_64"] + df_census["Female_35_64"]
df_census["Age_65plus"] = df_census["Male_65plus"] + df_census["Female_65plus"]

In [25]:
# demographic porportions 
df_census["Pct_Black"] = df_census["Black"] / df_census["TotalPop"] * 100
df_census["Pct_Hispanic"] = df_census["Hispanic"] / df_census["TotalPop"] * 100
df_census["Pct_Asian"] = df_census["Asian"] / df_census["TotalPop"] * 100
df_census["Pct_WhiteNonHisp"] = df_census["White_NonHisp"] / df_census["TotalPop"] * 100

In [26]:
# education proportions
df_census["Pct_BelowPoverty"] = df_census["BelowPoverty"] / df_census["TotalPop"] * 100
df_census["Pct_BachelorsPlus"] = df_census["BachelorsOrHigher"] / df_census["TotalPop"] * 100

#unemployment + insurance proportions
df_census["Pct_Unemployed"] = (df_census["Unemployed"] / df_census["LaborForce"]) * 100
df_census["Pct_Uninsured"] = (df_census["NoHealthInsurance"] / df_census["TotalPop"]) * 100

In [27]:
df_census.to_csv("data/processed/census_data_cleaned.csv", index=False)

### ME Data

In [28]:
df_me.head()

,Case Number,Date of Incident,Date of Death,Age,Gender,Race,Latino,Manner of Death,Primary Cause,Primary Cause Line A,Primary Cause Line B,Primary Cause Line C,Secondary Cause,Gun Related,Opioid Related,Cold Related,Heat Related,Commissioner District,Incident Address,Incident City,Incident Zip Code,longitude,latitude,location,Residence City,Residence Zip,OBJECTID,Chicago Ward,Chicago Community Area,COVID Related
0,ME2025-05835,11/04/2025 02:09:00 PM,11/04/2025 01:57:00 PM,57.0,Male,Black,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,2.0,"901 WEST 63RD ST., UNIT 324",CHICAGO,60621,-87.647293,41.779635,"(41.77963481, -87.64729284)",Chicago,60621,92724,16.0,ENGLEWOOD,False
1,ME2025-05834,09/24/2025 01:48:00 PM,11/04/2025 12:20:00 PM,55.0,Female,White,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,7.0,4803 S LAFLIN ST FLOOR ONE UNIT,CHICAGO,60609,-87.662232,41.806638,"(41.806638, -87.662232)",Chicago,60609,92725,20.0,NEW CITY,False
2,ME2025-05833,11/04/2025 12:00:00 AM,11/04/2025 12:40:00 PM,69.0,Female,White,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,1.0,2737 W HIRSCH STREET 2ND FLR,CHICAGO,60622,-87.696022,41.906421,"(41.906421, -87.6960225)",Chicago,60612,92720,26.0,WEST TOWN,False
3,ME2025-05832,11/04/2025 01:12:00 PM,11/04/2025 12:40:00 PM,75.0,Male,White,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,1.0,2737 W HIRSCH STREET 2ND FLR,CHICAGO,60622,-87.696022,41.906421,"(41.906421, -87.6960225)",Chicago,60622,92719,26.0,WEST TOWN,False
4,ME2025-05831,11/04/2025 01:41:00 PM,11/04/2025 09:48:00 AM,30.0,Male,Black,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,5.0,1722 W. 170TH,HAZEL CREST,60429,-87.659707,41.581080,"(41.58108, -87.6597075)",Hazel Crest,60429,92722,NaN,NaN,False


In [29]:
df_me.dtypes

Case Number                object
Date of Incident           object
Date of Death              object
Age                       float64
Gender                     object
Race                       object
Latino                       bool
Manner of Death            object
Primary Cause              object
Primary Cause Line A       object
Primary Cause Line B       object
Primary Cause Line C       object
Secondary Cause            object
Gun Related                object
Opioid Related             object
Cold Related                 bool
Heat Related                 bool
Commissioner District     float64
Incident Address           object
Incident City              object
Incident Zip Code          object
longitude                 float64
latitude                  float64
location                   object
Residence City             object
Residence Zip              object
OBJECTID                    int64
Chicago Ward              float64
Chicago Community Area     object
COVID Related 

In [30]:
## census data is from 2019-2023, so need to filter the ME data to match that

In [31]:
df_me["Updated_Date_of_Death"] = pd.to_datetime(df_me['Date of Death'], format='%m/%d/%Y %I:%M:%S %p')

In [32]:
df_me.columns

Index(['Case Number', 'Date of Incident', 'Date of Death', 'Age', 'Gender',
       'Race', 'Latino', 'Manner of Death', 'Primary Cause',
       'Primary Cause Line A', 'Primary Cause Line B', 'Primary Cause Line C',
       'Secondary Cause', 'Gun Related', 'Opioid Related', 'Cold Related',
       'Heat Related', 'Commissioner District', 'Incident Address',
       'Incident City', 'Incident Zip Code', 'longitude', 'latitude',
       'location', 'Residence City', 'Residence Zip', 'OBJECTID',
       'Chicago Ward', 'Chicago Community Area', 'COVID Related',
       'Updated_Date_of_Death'],
      dtype='object')

In [33]:
df_me["Year_of_Death"] = df_me["Updated_Date_of_Death"].dt.year
df_me.head()

,Case Number,Date of Incident,Date of Death,Age,Gender,Race,Latino,Manner of Death,Primary Cause,Primary Cause Line A,Primary Cause Line B,Primary Cause Line C,Secondary Cause,Gun Related,Opioid Related,Cold Related,Heat Related,Commissioner District,Incident Address,Incident City,Incident Zip Code,longitude,latitude,location,Residence City,Residence Zip,OBJECTID,Chicago Ward,Chicago Community Area,COVID Related,Updated_Date_of_Death,Year_of_Death
0,ME2025-05835,11/04/2025 02:09:00 PM,11/04/2025 01:57:00 PM,57.0,Male,Black,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,2.0,"901 WEST 63RD ST., UNIT 324",CHICAGO,60621,-87.647293,41.779635,"(41.77963481, -87.64729284)",Chicago,60621,92724,16.0,ENGLEWOOD,False,2025-11-04 13:57:00,2025.0
1,ME2025-05834,09/24/2025 01:48:00 PM,11/04/2025 12:20:00 PM,55.0,Female,White,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,7.0,4803 S LAFLIN ST FLOOR ONE UNIT,CHICAGO,60609,-87.662232,41.806638,"(41.806638, -87.662232)",Chicago,60609,92725,20.0,NEW CITY,False,2025-11-04 12:20:00,2025.0
2,ME2025-05833,11/04/2025 12:00:00 AM,11/04/2025 12:40:00 PM,69.0,Female,White,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,1.0,2737 W HIRSCH STREET 2ND FLR,CHICAGO,60622,-87.696022,41.906421,"(41.906421, -87.6960225)",Chicago,60612,92720,26.0,WEST TOWN,False,2025-11-04 12:40:00,2025.0
3,ME2025-05832,11/04/2025 01:12:00 PM,11/04/2025 12:40:00 PM,75.0,Male,White,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,1.0,2737 W HIRSCH STREET 2ND FLR,CHICAGO,60622,-87.696022,41.906421,"(41.906421, -87.6960225)",Chicago,60622,92719,26.0,WEST TOWN,False,2025-11-04 12:40:00,2025.0
4,ME2025-05831,11/04/2025 01:41:00 PM,11/04/2025 09:48:00 AM,30.0,Male,Black,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,5.0,1722 W. 170TH,HAZEL CREST,60429,-87.659707,41.581080,"(41.58108, -87.6597075)",Hazel Crest,60429,92722,NaN,NaN,False,2025-11-04 09:48:00,2025.0


In [34]:
df_me = df_me[(df_me["Year_of_Death"] >= 2019) & (df_me["Year_of_Death"] <= 2023)]
df_me.head()

,Case Number,Date of Incident,Date of Death,Age,Gender,Race,Latino,Manner of Death,Primary Cause,Primary Cause Line A,Primary Cause Line B,Primary Cause Line C,Secondary Cause,Gun Related,Opioid Related,Cold Related,Heat Related,Commissioner District,Incident Address,Incident City,Incident Zip Code,longitude,latitude,location,Residence City,Residence Zip,OBJECTID,Chicago Ward,Chicago Community Area,COVID Related,Updated_Date_of_Death,Year_of_Death
3552,ME2025-02283,04/21/2023 02:08:00 PM,07/25/2023 12:00:00 AM,42.0,Male,Black,False,ACCIDENT,COMPLICATIONS OF DROWNING,COMPLICATIONS OF DROWNING,NaN,NaN,"HYPERTENSIVE CARDIOVASCULAR DISEASE, CHRONIC S...",False,False,False,False,1.0,100 N Central Park Avenue,CHICAGO,60624,-87.716075,41.882688,"(41.882688, -87.7160745)",Elmwood Park,60707,89165,28.0,EAST GARFIELD PARK,False,2023-07-25 00:00:00,2023.0
4254,ME2025-01581,03/10/2025 12:00:00 AM,05/03/2023 11:36:00 PM,84.0,Male,Black,False,NATURAL,MALIGNANT NEOPLASM OF THE URINARY BLADDER,MALIGNANT NEOPLASM OF THE URINARY BLADDER,NaN,NaN,NaN,False,False,False,False,5.0,230 E. 11th St.,CHICAGO HEIGHTS,60411,-87.625580,41.511357,"(41.511357, -87.6255795)",NaN,NaN,88467,NaN,NaN,False,2023-05-03 23:36:00,2023.0
4257,ME2025-01578,NaN,05/15/2023 06:29:00 AM,76.0,Male,White,False,NATURAL,HYPERTENSIVE ARTERIOSCLEROTIC CARDIOVASCULAR D...,HYPERTENSIVE ARTERIOSCLEROTIC CARDIOVASCULAR D...,NaN,NaN,DIABETES MELLITUS,False,False,False,False,NaN,NaN,UNKNOWN,NaN,NaN,NaN,NaN,NaN,NaN,88476,NaN,NaN,False,2023-05-15 06:29:00,2023.0
4258,ME2025-01577,03/06/2022 08:11:00 PM,03/06/2022 07:20:00 PM,71.0,Male,White,True,NATURAL,CHRONIC ETHANOLISM,CHRONIC ETHANOLISM,NaN,NaN,DIABETES MELLITUS,False,False,False,False,7.0,2551 W. CERMAK ROAD,CHICAGO,60608,-87.690001,41.850558,"(41.850558, -87.6900015)",Chicago,60623,88470,28.0,SOUTH LAWNDALE,False,2022-03-06 19:20:00,2022.0
12057,ME2024-00980,11/30/2023 02:23:00 PM,12/01/2023 07:52:00 AM,87.0,Female,Asian,False,NATURAL,SEPSIS. BRONCHOPNEUMONIA,SEPSIS,BRONCHOPNEUMONIA,NaN,HYPERTENSIVE AND ATHEROSCLEROTIC CARDIOVASCULA...,False,False,False,False,NaN,8110 Lincoln Ave. #2C,CHICAGO,60646,NaN,NaN,NaN,Skokie,60077,80661,NaN,NaN,False,2023-12-01 07:52:00,2023.0


In [35]:
len(df_me)
# 53086 deaths between 2019 and 2023

53086

In [36]:
# create columns for deaths related to drugs 
fent_names = "FENTANYL| 4-ANILINO-N-PHENETHYLPIPERIDINE | 4-ANPP | Acetyl-alphamethyl-fentanyl | Alfentanil | Alpha-methylfentanyl | 4ANPP | 4FIBF | Alpha-methylthiofentanyl | Beta-hydroxyfentanyl | Beta-hydroxy-3 methylfentanyl | 3-methylfentanyl | 3-methylthio-fentanyl | Para-fluoro-fentanyl | Remifentanil | Sufentanil | Thiofentanyl | Carfentanil | 2-furanoylfentanyl | Furanylfentanyl | 4-anilino-N-phenethylpiperidine | 4-Fluorofentanyl | ACETYLFENTANYL | ACRYLFENTANYL | Butyrfentanyl | Lofentanil | Valerylfentanyl | Isobutyrylfentanyl"
df_me['contanins_fentanyl'] = df_me['Primary Cause'].str.contains(fent_names, case=False, na=False)

In [37]:
df_me['contains_cocaine'] = df_me['Primary Cause'].str.contains("cocaine|Benzoylmethylecgonine", case=False, na=False)
df_me["contains_heroin"] =df_me['Primary Cause'].str.contains('HEROIN', case=False, na=False)

In [38]:
df_me["Incident Zip Code"] = df_me["Incident Zip Code"].astype(str).str.extract(r"(\d{5})")
df_me["Incident Zip Code"] = df_me["Incident Zip Code"].str.zfill(5)
df_me.head()

,Case Number,Date of Incident,Date of Death,Age,Gender,Race,Latino,Manner of Death,Primary Cause,Primary Cause Line A,Primary Cause Line B,Primary Cause Line C,Secondary Cause,Gun Related,Opioid Related,Cold Related,Heat Related,Commissioner District,Incident Address,Incident City,Incident Zip Code,longitude,latitude,location,Residence City,Residence Zip,OBJECTID,Chicago Ward,Chicago Community Area,COVID Related,Updated_Date_of_Death,Year_of_Death,contanins_fentanyl,contains_cocaine,contains_heroin
3552,ME2025-02283,04/21/2023 02:08:00 PM,07/25/2023 12:00:00 AM,42.0,Male,Black,False,ACCIDENT,COMPLICATIONS OF DROWNING,COMPLICATIONS OF DROWNING,NaN,NaN,"HYPERTENSIVE CARDIOVASCULAR DISEASE, CHRONIC S...",False,False,False,False,1.0,100 N Central Park Avenue,CHICAGO,60624,-87.716075,41.882688,"(41.882688, -87.7160745)",Elmwood Park,60707,89165,28.0,EAST GARFIELD PARK,False,2023-07-25 00:00:00,2023.0,False,False,False
4254,ME2025-01581,03/10/2025 12:00:00 AM,05/03/2023 11:36:00 PM,84.0,Male,Black,False,NATURAL,MALIGNANT NEOPLASM OF THE URINARY BLADDER,MALIGNANT NEOPLASM OF THE URINARY BLADDER,NaN,NaN,NaN,False,False,False,False,5.0,230 E. 11th St.,CHICAGO HEIGHTS,60411,-87.625580,41.511357,"(41.511357, -87.6255795)",NaN,NaN,88467,NaN,NaN,False,2023-05-03 23:36:00,2023.0,False,False,False
4257,ME2025-01578,NaN,05/15/2023 06:29:00 AM,76.0,Male,White,False,NATURAL,HYPERTENSIVE ARTERIOSCLEROTIC CARDIOVASCULAR D...,HYPERTENSIVE ARTERIOSCLEROTIC CARDIOVASCULAR D...,NaN,NaN,DIABETES MELLITUS,False,False,False,False,NaN,NaN,UNKNOWN,NaN,NaN,NaN,NaN,NaN,NaN,88476,NaN,NaN,False,2023-05-15 06:29:00,2023.0,False,False,False
4258,ME2025-01577,03/06/2022 08:11:00 PM,03/06/2022 07:20:00 PM,71.0,Male,White,True,NATURAL,CHRONIC ETHANOLISM,CHRONIC ETHANOLISM,NaN,NaN,DIABETES MELLITUS,False,False,False,False,7.0,2551 W. CERMAK ROAD,CHICAGO,60608,-87.690001,41.850558,"(41.850558, -87.6900015)",Chicago,60623,88470,28.0,SOUTH LAWNDALE,False,2022-03-06 19:20:00,2022.0,False,False,False
12057,ME2024-00980,11/30/2023 02:23:00 PM,12/01/2023 07:52:00 AM,87.0,Female,Asian,False,NATURAL,SEPSIS. BRONCHOPNEUMONIA,SEPSIS,BRONCHOPNEUMONIA,NaN,HYPERTENSIVE AND ATHEROSCLEROTIC CARDIOVASCULA...,False,False,False,False,NaN,8110 Lincoln Ave. #2C,CHICAGO,60646,NaN,NaN,NaN,Skokie,60077,80661,NaN,NaN,False,2023-12-01 07:52:00,2023.0,False,False,False


In [39]:
df_me.head()

,Case Number,Date of Incident,Date of Death,Age,Gender,Race,Latino,Manner of Death,Primary Cause,Primary Cause Line A,Primary Cause Line B,Primary Cause Line C,Secondary Cause,Gun Related,Opioid Related,Cold Related,Heat Related,Commissioner District,Incident Address,Incident City,Incident Zip Code,longitude,latitude,location,Residence City,Residence Zip,OBJECTID,Chicago Ward,Chicago Community Area,COVID Related,Updated_Date_of_Death,Year_of_Death,contanins_fentanyl,contains_cocaine,contains_heroin
3552,ME2025-02283,04/21/2023 02:08:00 PM,07/25/2023 12:00:00 AM,42.0,Male,Black,False,ACCIDENT,COMPLICATIONS OF DROWNING,COMPLICATIONS OF DROWNING,NaN,NaN,"HYPERTENSIVE CARDIOVASCULAR DISEASE, CHRONIC S...",False,False,False,False,1.0,100 N Central Park Avenue,CHICAGO,60624,-87.716075,41.882688,"(41.882688, -87.7160745)",Elmwood Park,60707,89165,28.0,EAST GARFIELD PARK,False,2023-07-25 00:00:00,2023.0,False,False,False
4254,ME2025-01581,03/10/2025 12:00:00 AM,05/03/2023 11:36:00 PM,84.0,Male,Black,False,NATURAL,MALIGNANT NEOPLASM OF THE URINARY BLADDER,MALIGNANT NEOPLASM OF THE URINARY BLADDER,NaN,NaN,NaN,False,False,False,False,5.0,230 E. 11th St.,CHICAGO HEIGHTS,60411,-87.625580,41.511357,"(41.511357, -87.6255795)",NaN,NaN,88467,NaN,NaN,False,2023-05-03 23:36:00,2023.0,False,False,False
4257,ME2025-01578,NaN,05/15/2023 06:29:00 AM,76.0,Male,White,False,NATURAL,HYPERTENSIVE ARTERIOSCLEROTIC CARDIOVASCULAR D...,HYPERTENSIVE ARTERIOSCLEROTIC CARDIOVASCULAR D...,NaN,NaN,DIABETES MELLITUS,False,False,False,False,NaN,NaN,UNKNOWN,NaN,NaN,NaN,NaN,NaN,NaN,88476,NaN,NaN,False,2023-05-15 06:29:00,2023.0,False,False,False
4258,ME2025-01577,03/06/2022 08:11:00 PM,03/06/2022 07:20:00 PM,71.0,Male,White,True,NATURAL,CHRONIC ETHANOLISM,CHRONIC ETHANOLISM,NaN,NaN,DIABETES MELLITUS,False,False,False,False,7.0,2551 W. CERMAK ROAD,CHICAGO,60608,-87.690001,41.850558,"(41.850558, -87.6900015)",Chicago,60623,88470,28.0,SOUTH LAWNDALE,False,2022-03-06 19:20:00,2022.0,False,False,False
12057,ME2024-00980,11/30/2023 02:23:00 PM,12/01/2023 07:52:00 AM,87.0,Female,Asian,False,NATURAL,SEPSIS. BRONCHOPNEUMONIA,SEPSIS,BRONCHOPNEUMONIA,NaN,HYPERTENSIVE AND ATHEROSCLEROTIC CARDIOVASCULA...,False,False,False,False,NaN,8110 Lincoln Ave. #2C,CHICAGO,60646,NaN,NaN,NaN,Skokie,60077,80661,NaN,NaN,False,2023-12-01 07:52:00,2023.0,False,False,False


In [40]:
df_me.isna().sum()
# 1295 missing zip code 

Case Number                   0
Date of Incident           1693
Date of Death                 0
Age                         170
Gender                      116
Race                        231
Latino                        0
Manner of Death             252
Primary Cause                14
Primary Cause Line A         14
Primary Cause Line B      33920
Primary Cause Line C      50234
Secondary Cause           29868
Gun Related                 254
Opioid Related              254
Cold Related                  0
Heat Related                  0
Commissioner District      6914
Incident Address            822
Incident City               958
Incident Zip Code          1295
longitude                  6909
latitude                   6909
location                   6909
Residence City             1169
Residence Zip              1151
OBJECTID                      0
Chicago Ward              24105
Chicago Community Area    24105
COVID Related                 2
Updated_Date_of_Death         0
Year_of_

In [41]:
df_me = df_me.dropna(subset=["Incident Zip Code"])
len(df_me)

51791

In [42]:
df_me.isna().sum()


Case Number                   0
Date of Incident           1266
Date of Death                 0
Age                         160
Gender                      106
Race                        216
Latino                        0
Manner of Death             241
Primary Cause                14
Primary Cause Line A         14
Primary Cause Line B      33387
Primary Cause Line C      49101
Secondary Cause           29409
Gun Related                 243
Opioid Related              243
Cold Related                  0
Heat Related                  0
Commissioner District      5656
Incident Address            155
Incident City                78
Incident Zip Code             0
longitude                  5651
latitude                   5651
location                   5651
Residence City             1062
Residence Zip              1051
OBJECTID                      0
Chicago Ward              22836
Chicago Community Area    22836
COVID Related                 2
Updated_Date_of_Death         0
Year_of_

In [43]:
# AFTER CLEANING, save ME data to this!!!


# df_me = pd.read_csv('data/raw/cook_county_medical_examiner.csv')
# df_census = pd.read_csv('data/raw/census_tract_data.csv')

# # Save processed copy and compute checksum
# df_me.to_csv('data/processed/me_clean.csv', index=False)
# with open('data/processed/me_clean.csv', 'rb') as f:
#     checksum = hashlib.sha256(f.read()).hexdigest()
# print("Checksum:", checksum)